<a href="https://colab.research.google.com/github/oksyassine/Find-the-Intruder/blob/master/NLP_Skip_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing The Workspace

In [0]:
!pip install allennlp > /dev/null
!wget http://mattmahoney.net/dc/text8.zip
!unzip text8.zip -d data/
!wget https://fh295.github.io/SimLex-999.zip
!unzip SimLex-999.zip -d data/

We import all the necessary modules and define some constants used in this program

In [0]:
from collections import Counter

import torch
import torch.optim as optim
import numpy as np
import math
#import random
from allennlp.data.instance import Instance
from allennlp.data.fields import LabelField
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.data.iterators import BasicIterator
from allennlp.modules.token_embedders import Embedding
from allennlp.training.trainer import Trainer
from allennlp.training.checkpointer import Checkpointer
from overrides import overrides
from scipy.stats import spearmanr,pearsonr
from torch.nn import CosineSimilarity
from torch.nn import functional

CUDA_DEVICE = 0

#Defining the DatasetReader and Preparing the data

First, we need to implement a dataset reader that reads a plain text corpus and turns it into a set of instances that can be consumed by the Skip-gram model. 



In [0]:
@DatasetReader.register("skip_gram")
class SkipGramReader(DatasetReader):
    def __init__(self, window_size=2, lazy=False, vocab: Vocabulary=None):
        #if we set "lazy" variable to True, DatasetReader will create and yield up instances as needed rather than all at once.
        super().__init__(lazy=lazy)
        #window_size is the maximum distance between the focus word and its contextual neighbors
        self.window_size = window_size
        self.reject_probs = None
        if vocab:
            self.reject_probs = {}#it will contain a pre-computed dict of probabilities for subsampling
            threshold = 6*1.e-3 # freq_max of rare Words not subsampled
            token_counts = vocab._retained_counter['token_in'] # collections.defaultdict # would be set only if instances were used for vocabulary construction
            total_counts = sum(token_counts.values())
            
            for _, token in vocab.get_index_to_token_vocabulary('token_in').items():
                counts = token_counts[token]#Count the number of times a word is reapeated in the text corpus
                if counts > 0:
                    normalized_counts = counts / total_counts
                    reject_prob = 1. - math.sqrt(threshold / normalized_counts)
                    reject_prob = max(0., reject_prob)
                else:
                    reject_prob = 0.
                self.reject_probs[token] = reject_prob

    def _subsample_tokens(self, tokens):
        """Given a list of tokens, runs sub-sampling.

        Returns a new list of tokens where rejected tokens are replaced by Nones.
        """
        new_tokens = []
        for token in tokens:
            reject_prob = self.reject_probs.get(token, 0.)
            if 0 < reject_prob:
                new_tokens.append(None)
            else:
                new_tokens.append(token)

        return new_tokens

    @overrides
    def _read(self, file_path: str):
        with open(cached_path(file_path), "r") as text_file:
            for line in text_file:
                tokens = line.strip().split(' ') #list of tokens
                tokens = tokens[:9750000] # Due To limitation on the RAM Memory, we should truncate the training data first,
                                          # by using only the first nine million tokens
                if self.reject_probs:
                    tokens = self._subsample_tokens(tokens)
                    print(tokens[:200])  # for debugging

                for i, token in enumerate(tokens):
                    if token is None:
                        continue

                    token_in = LabelField(token, label_namespace='token_in')

                    for j in range(i - self.window_size, i + self.window_size + 1):
                        if j < 0 or i == j or j > len(tokens) - 1:
                            continue

                        if tokens[j] is None:
                            continue

                        token_out = LabelField(tokens[j], label_namespace='token_out') # Labelfield
                        yield Instance({'token_in': token_in, 'token_out': token_out}) # Instance


We use the SkipGramReader as follows:

In [0]:
reader = SkipGramReader()
text8 = reader.read('data/text8')
#Once we've read the dataset, we use it to create our vocabulary
vocab = Vocabulary.from_instances(text8, min_count={'token_in': 2, 'token_out': 2})

del(text8)
#reading the dataset with Vocabulary to sub-sample the frequent words 
reader = SkipGramReader(vocab=vocab)
text8 = reader.read('data/text8')

0it [00:00, ?it/s]Your label namespace was 'token_in'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
Your label namespace was 'token_out'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
38415767it [04:13, 205188.48it/s]

Note that we are using min_count, which sets the lower bound on the number of occurrences for each token. Also, let’s define the iterator we use for training.
A very basic iterator that takes a dataset, possibly shuffles it, and creates fixed sized batches.

In [0]:
BATCH_SIZE = 256 #batch_size specifies the size of the batch (the number of instances in a batch)
iterator = BasicIterator(batch_size=BATCH_SIZE)
iterator.index_with(vocab)

Then, you need to initialize an Embedding instance, which takes care of converting IDs to embeddings. The size (dimension) of the embeddings is determined by EMBEDDING_DIM.



In [0]:
EMBEDDING_DIM = 300
embedding_in = Embedding(num_embeddings=vocab.get_vocab_size('token_in'),
                         embedding_dim=EMBEDDING_DIM)
if CUDA_DEVICE > -1:
  embedding_in = embedding_in.to(CUDA_DEVICE) # we are able now to use GPU 0 

# Defining the skip-gram Model

Finally, you need to implement the body of the Skip-gram model.


In [0]:
#1 we implement the Skip-gram model
class SkipGramModel(Model):
    def __init__(self, vocab, embedding_in, cuda_device=-1):
        super().__init__(vocab)
#2 Embedding object is passed from outside rather than defined inside
        self.embedding_in = embedding_in
#3 this create a linear layer(we don't need biases)       
        self.linear = torch.nn.Linear(
            in_features=EMBEDDING_DIM,#size of the input vector
            out_features=vocab.get_vocab_size('token_out'),#size of the output vector
            bias=False)      
        if cuda_device > -1:
            self.linear = self.linear.to(cuda_device)
#4 Body of neural network computation is implemented in forward
    def forward(self, token_in, token_out):
#5 convert input tensors(word IDs) to word embeddings     
        embedded_in = self.embedding_in(token_in)
#6 Apply linear layer     
        logits = self.linear(embedded_in)
#7 This combines softmax and loss ompute the loss in a single function .c
        loss = functional.cross_entropy(logits, token_out)
        return {'loss': loss}

In [0]:
model = SkipGramModel(vocab=vocab,
                      embedding_in=embedding_in,
                      cuda_device=CUDA_DEVICE)

Here, we are using one type of optimizer called Adam,we can initialize an optimizer using PyTorch as follow:

In [0]:
optimizer = optim.Adam(model.parameters())

In order to avoid the situation of getting disconnected from the hosted runtime and losing all your progress on the training step, you should mount your Google Drive in order to save your checkpoints inside, then you can start from the last checkpoint saved at anytime.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
checkpoint = Checkpointer(serialization_dir="/content/drive/My Drive/_AI.NLP/check6-3/")

# Training our own classifier

Now we are ready to train our model.We assume that we have already defined and initialized the model.


In [0]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=text8,
                  patience=3,
                  num_epochs=30,
                  checkpointer=checkpoint,
                  cuda_device=CUDA_DEVICE)

When we launch the following line, it will print a progress bar for each epoch that includes the "loss" metric. If our model is good, the loss should go down as we train.

In [0]:
trainer.train()

loss: 9.6876 ||:   3%|▎         | 1916/70862 [04:59<2:47:34,  6.86it/s]

#  Evaluating and Exploring the Model Results 

After the training is finished, we can get related words using the method get_related .

In [0]:
def get_related(token: str, embedding: Model, vocab: Vocabulary, num_related: int = 20):
    """Given a token, return a list of top 20 most similar words to the token."""
    token_id = vocab.get_token_index(token, 'token_in')
    token_vec = embedding.weight[token_id]#A pre-initialization weight matrix for the embedding lookup, allowing the use of pretrained vectors.
    cosine = CosineSimilarity(dim=0)  #we do this to be able calculate simple cosine similarity between 2 vectors
    sims = Counter()

    for index, token in vocab.get_index_to_token_vocabulary('token_in').items():
        # Cosine similarity of our token vector with every other word vector in the vocabulary
        sim = cosine(token_vec, embedding.weight[index]).item() 
        sims[token] = sim #save the value of cosine similarity

    return sims.most_common(num_related)

If we run this for words “singer” and “december” and "internet" we get the lists of related words. Although we can see some words that are not related to the query word, overall the results look good.

In [0]:
print(get_related('december', embedding_in, vocab))

In [0]:
print(get_related('flower', embedding_in, vocab))

In [0]:
print(get_related('design', embedding_in, vocab))

Let's measure the quality of word embeddings by using the simlex999 dataset


In [0]:
def read_simlex999():
    simlex999 = []
    with open('data/SimLex-999/SimLex-999.txt') as f: #open the test file SimLex-999.txt
        next(f)
        for line in f:
            fields = line.strip().split('\t') #there is a string who takes values from each line
            word1, word2, _, sim = fields[:4] #extract only the four first columns in the file SimLex-999.txt
            sim = float(sim) #cast value
            simlex999.append((word1, word2, sim)) #add each couple of words with their similarity into simlex999
    return simlex999

def evaluate_embeddings(embedding, vocab: Vocabulary):
    cosine = CosineSimilarity(dim=0)
    simlex999 = read_simlex999()
    sims_pred = []
    oov_count = 0
    for word1, word2, sim in simlex999:
        word1_id = vocab.get_token_index(word1, 'token_in') #word1_id takes the ID of the word 1.
        if word1_id == 1: # word_ID==1  means that that the word is out of vocabulary OOV
            sims_pred.append(0.)
            oov_count += 1 
            continue
        word2_id = vocab.get_token_index(word2, 'token_in') #word2_id takes the ID of the word 2
        if word2_id == 1:
            sims_pred.append(0.)
            oov_count += 1
            continue

        sim_pred = cosine(embedding.weight[word1_id],
                          embedding.weight[word2_id]).item() #Calculate the CosineSimilarity between word1 and word2 and charge this in sim_pred.
        sims_pred.append(sim_pred)

    assert len(sims_pred) == len(simlex999) # Assertion de l'egalité de longueur de sims_pred et simlex999
    print('# of OOV words: {} / {}'.format(oov_count, len(simlex999)))
    print(pearsonr(sims_pred, [sim for _, _, sim in simlex999]))
    return spearmanr(sims_pred, [sim for _, _, sim in simlex999]) # compare two sets of similarities and calculate how they are related, it's called spearman's correlation
    #compare two sets of similarities and calculate how they are related.
    #Calculates a Spearman rank-order correlation coefficient and the p-value to test for non-correlation.
    """scipy.stats.spearmanr(a, b=None, axis=0)[source]
Calculates a Spearman rank-order correlation coefficient and the p-value to test for non-correlation."""

In [0]:
rho = evaluate_embeddings(embedding_in, vocab)
print('simlex999 speareman correlation: {}'.format(rho))

# Export Our Model

In [0]:
def write_embeddings(embedding: Embedding, file_path, vocab: Vocabulary):
    with open(file_path, mode='w') as f:
        words=vocab.get_index_to_token_vocabulary('token_in').items()
        print(len(words))
        f.write('{} {}\n'.format(len(words),EMBEDDING_DIM))#we write number of words and embedding dimension
        for index, token in words: #loop through both keys and values, by using the items()
            values = ['{:.10f}'.format(val) for val in embedding.weight[index]]#write each value as a number with 10 decimals
            f.write(' '.join([token] + values))
            f.write('\n')

In [0]:
write_embeddings(embedding_in, 'drive/My Drive/_AI.NLP/embed100000.txt', vocab)




#Word Visualization




In [0]:
#!wget -O embed30.txt https://skipgram.azurewebsites.net

We  can use t-SNE to visualize word embeddings

In [0]:
from sklearn.manifold import TSNE #import TSNE technic from manifold
import matplotlib.pyplot as plt
# Reading a mixture of text and numbers
def read_model(file_path):
    with open(file_path) as f:
        i=0
        for line in f.readlines()[6:]:
            row = line.split()#for each line, split the line into :
            word = row[0] # word in row[0]
            vec = [float(x) for x in row[1:]]#and vector for the rest of the line in row[1:].
            yield (word, vec) #return every iteration ( yield : generator )
            i+=1#passing to the next ligne
            if i == 1100:
                break

In [0]:
words = []
vectors = []
for word, vec in read_model('drive/My Drive/_AI.NLP/vectors/embedfull6-3.txt'):#pour tous les 1100 couples de (word,vec) retourné
    words.append(word)#add each word to  words's array.
    vectors.append(vec)#parallèlement, ajouter chaque vecteur dans la liste vectors

modele = TSNE(n_components=2, init='pca', random_state=0) #Fixing Dimension of the embedded space et 2 ,PCA Initialization of embedding,the random number generator is the RandomState instance used by np.random.
coordinates = modele.fit_transform(vectors)#  Performs the calculations and returns the 2 principal components(Fit to data, then transform it.)

plt.figure(figsize=(50, 50))

for word, xy in zip(words, coordinates): #parallele aggregation of elements from two iterables(words,coordinates)
    plt.scatter(xy[0], xy[1])
    plt.annotate(word, #add annotations to our plot
                  xy=(xy[0], xy[1]),
                  xytext=(2, 2),
                  textcoords='offset points')
    

plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.show()

In [0]:
!cp embed30.txt drive/My\ Drive/_AI.NLP

In [0]:
!pip install -U gensim > /dev/null

In [0]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
modell = KeyedVectors.load_word2vec_format('drive/My Drive/_AI.NLP/vectors/embedfull6-3.txt')

In [0]:
modell.most_similar('muslim')

[('islamic', 0.5333488583564758),
 ('muslims', 0.4839276075363159),
 ('arab', 0.46515345573425293),
 ('christians', 0.4155333638191223),
 ('shia', 0.4111873507499695),
 ('islam', 0.407677561044693),
 ('sunni', 0.37980395555496216),
 ('jews', 0.3720093369483948),
 ('jewish', 0.3604156970977783),
 ('sadiq', 0.3567056357860565)]

In [0]:
modell.most_similar('singer')

[('musician', 0.6285589933395386),
 ('actress', 0.5955820083618164),
 ('actor', 0.5759903788566589),
 ('songwriter', 0.5299771428108215),
 ('guitarist', 0.5003552436828613),
 ('american', 0.4894171953201294),
 ('composer', 0.4785977303981781),
 ('comedian', 0.4512455463409424),
 ('eydie', 0.45047467947006226),
 ('writer', 0.4242362976074219)]

In [0]:
modell.most_similar('internet')

[('asiilasooq', 0.3801255524158478),
 ('tld', 0.3671312928199768),
 ('server', 0.36664992570877075),
 ('telnet', 0.3655414283275604),
 ('online', 0.36259526014328003),
 ('isps', 0.3623088300228119),
 ('mail', 0.3535529375076294),
 ('servers', 0.3529438078403473),
 ('network', 0.35159844160079956),
 ('web', 0.35117414593696594)]

In [0]:
modell.similarity('college','university')

0.7127381

In [0]:
# Which word from the given list doesn't go with the others?
modell.doesnt_match("breakfast cereal dinner lunch".split())
#> beetroot

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'cereal'

In [0]:
modell.doesnt_match("college university hospital school".split())

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'hospital'

In [0]:
modell.doesnt_match("python java perl hardware programming ruby javascript php html".split())

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'hardware'

In [0]:
#modell.evaluate_word_analogies(datapath("questions-words.txt"))

In [0]:
modell.evaluate_word_analogies(datapath("questions-words.txt"))[0]

0.18637414636769373

In [0]:
modell.similar_by_vector(modell.get_vector('father')-modell.get_vector('boy')+modell.get_vector('girl'))

[('father', 0.617018461227417),
 ('girl', 0.5638644695281982),
 ('mother', 0.4166046977043152),
 ('daughter', 0.39282089471817017),
 ('son', 0.34039145708084106),
 ('brother', 0.3313406705856323),
 ('wife', 0.3182443678379059),
 ('married', 0.3146499693393707),
 ('torlonia', 0.3022747039794922),
 ('childhood', 0.2773662805557251)]

In [0]:
modell.words_closer_than("king","queen")
# Get the words closer to w1 than w2
#modell.words_closer_than('king', 'kingdom'))
#> ['prince', 'queen', 'monarch']

['son', 'prince', 'kings']

In [0]:
modell.words_closer_than("queen","king")

['elizabeth']

In [0]:
modell.words_closer_than("team","player")

['league', 'teams']

In [0]:
modell.words_closer_than("soccer","team")

['football', 'hockey', 'basketball', 'rugby']